In [1]:
import pandas as pd
from tqdm import tqdm

df = pd.read_excel('../predict_result/0927到1210測試結果.xlsx')
print(df.shape)
df.head()

(1790, 47)


,45A,項次,電文代號,銀行,20,回傳代碼,回傳訊息,建檔日期,受益人,金額,...,EXPNO jac,EXPNO 產品名,EXPNO 開狀人,EXPNO 受益人,EXPNO 開狀銀行,EXPNO 產品名分數,EXPNO 開狀人分數,EXPNO 受益人分數,EXPNO 開狀銀行分數,與前案一致
0,48 MT PVC SUSPENSION RESIN GRADE S65AT THE RA...,8,mt700,16,0006MLC00034022,200,"{""MESSAGE"":"""",""CODE"":200}",2021/09/24 12:31:17,FORMOSA PLASTICS,"USD136080,00",...,1.066667,MASS PVC RESIN B-57,OJUS PETROCHEMICALS LLPC,FORMOSA PLASTICS CORPORATION,"ICICI BANK LTD.,\nSHALIMAR TOWER, 31/54 M.G.MA...",0.400000,0.0,0.666667,0.0,True
1,TETRAHYDROFURAN AT USD 5550 TOTAL AMOUNTUSD 1...,10,mt700,16,001LM01212660007,200,"{""MESSAGE"":"""",""CODE"":200}",2021/09/24 12:32:17,NAN YA PLASTICS,"USD111000,",...,0.750000,(鄰苯二甲酐)\nPHTHALIC ANHYDRIDE,AHMED SAEED AFIFI FACTORY CO FORRESINS LTD,NAN YA PLASTICS CORPORATION,"NATIONAL COMMERCIAL BANK, THE\n(HEAD OFFICE)",0.000000,0.0,0.750000,0.0,False
2,A 19800 MT OF TAIRILAC AG15A1HABS ABS RESINAT...,11,mt700,16,0393MLC00068522,200,"{""MESSAGE"":"""",""CODE"":200}",2021/09/24 12:33:05,FORMOSA CHEMICALS AND FIBRE,"USD736110,00",...,1.133333,ABS RESIN GRADE\nAG15A1-H\n丙烯-丁二烯-\n苯乙烯共聚合物,SHANTILAL MAHENDRAKUMAR AND COSHILPIN COMMERCI...,FORMOSA CHEMICALS AND FIBRE CORP,BANK OF MAHARASHTRA,0.333333,0.0,0.800000,0.0,True
3,80 MT PVC SUSPENSION RESIN GRADE S65S AT THE ...,12,mt700,16,0505121IM0000553,200,"{""MESSAGE"":"""",""CODE"":200}",2021/09/24 12:01:28,FORMOSA PLASTICS,"USD154710,00",...,1.066667,MASS PVC RESIN B-57,OJUS PETROCHEMICALS LLPC,FORMOSA PLASTICS CORPORATION,"ICICI BANK LTD.,\nSHALIMAR TOWER, 31/54 M.G.MA...",0.400000,0.0,0.666667,0.0,True
4,PVC SUSPENSION RESIN S70RQUANTITY 72 MT UNIT ...,13,mt700,16,0799M21SA0000380,200,"{""MESSAGE"":"""",""CODE"":200}",2021/09/24 12:01:04,FORMOSA PLASTICS,"USD152640,",...,1.066667,MASS PVC RESIN B-57,OJUS PETROCHEMICALS LLPC,FORMOSA PLASTICS CORPORATION,"ICICI BANK LTD.,\nSHALIMAR TOWER, 31/54 M.G.MA...",0.400000,0.0,0.666667,0.0,True


In [2]:
df.columns

Index(['45A', '項次', '電文代號', '銀行', '20', '回傳代碼', '回傳訊息', '建檔日期', '受益人', '金額',
       '推薦公司事業部', '是否已轉MIS', 'MEMO', 'from', '50', '59', '46A', '47A', '78',
       'LTADDRESS.1', 'memo', 'Unnamed: 5', '預測產品', '預測產品(取長度最長)', '預測產品使用方式',
       '根據產品預測部門', '根據產品預測代號', '預測開狀人', '利用公司名稱預測公司代號', 'DIVSION',
       'DIVSION預測代號', '集成預測代號', '銀行輸入', '開狀銀行', '正確與否', '錯誤原因', 'EXPNO',
       'EXPNO jac', 'EXPNO 產品名', 'EXPNO 開狀人', 'EXPNO 受益人', 'EXPNO 開狀銀行',
       'EXPNO 產品名分數', 'EXPNO 開狀人分數', 'EXPNO 受益人分數', 'EXPNO 開狀銀行分數', '與前案一致'],
      dtype='object')

In [3]:
def save_color_df(df,save_path,x_cols=['45A','50','59','銀行輸入'],y_cols=['預測產品(取長度最長)','預測開狀人','受益人','開狀銀行']):
    # 建立writer
    writer = pd.ExcelWriter(save_path, engine='xlsxwriter')
    # 將 df 第一個 row 變成欄位名稱
    new_df = pd.DataFrame()
    for i in df.columns:
        new_df[i] = [i] + df[i].values.tolist() 
    df = new_df
    # 存檔
    df.to_excel(writer, sheet_name='Sheet1', header=False, index=False)
    # 參數設定
    workbook  = writer.book
    worksheet = writer.sheets['Sheet1']
    cell_format_red = workbook.add_format({'font_color': 'red'})
    cell_format_default = workbook.add_format({'bold': False})
    worksheet.write_row('A1',df.columns.tolist())
    # 定義反紅功能函數
    def add_word_color(df,x_col,y_col):
        for row in range(0,len(df)):
            word = df.iloc[row,:][y_col]
            detect_col_idx = df.columns.tolist().index(x_col)
            try:
                # 1st case, wrong word is at the start and there is additional text
                if (df.iloc[row,detect_col_idx].index(word) == 0) \
                and (len(df.iloc[row,detect_col_idx]) != len(word)):
                    worksheet.write_rich_string(row,detect_col_idx,cell_format_red,
                        word,cell_format_default,df.iloc[row,detect_col_idx][len(word):])

                # 2nd case, wrong word is at the middle of the string
                elif (df.iloc[row,detect_col_idx].index(word) > 0) \
                and (df.iloc[row,detect_col_idx].index(word) != len(df.iloc[row,detect_col_idx])-len(word)) \
                and ('Typo:' not in df.iloc[row,detect_col_idx]):
                    starting_point = df.iloc[row,detect_col_idx].index(word)
                    worksheet.write_rich_string(row, detect_col_idx, cell_format_default,
                                        df.iloc[row,detect_col_idx][0:starting_point],
                                        cell_format_red, word, cell_format_default,
                                        df.iloc[row,detect_col_idx][starting_point+len(word):])

                # 3rd case, wrong word is at the end of the string
                elif (df.iloc[row,detect_col_idx].index(word) > 0) \
                and (df.iloc[row,detect_col_idx].index(word) == len(df.iloc[row,detect_col_idx])-len(word)):
                    starting_point = df.iloc[row,detect_col_idx].index(word)
                    worksheet.write_rich_string(row, detect_col_idx, cell_format_default,
                                                df.iloc[row,detect_col_idx][0:starting_point],
                                                cell_format_red, word)

                # 4th case, wrong word is the only one in the string
                elif (df.iloc[row,detect_col_idx].index(word) == 0) \
                and (len(df.iloc[row,detect_col_idx]) == len(word)):
                    worksheet.write(row, detect_col_idx, word, cell_format_red)

            except ValueError:
                continue
    
    # 執行多次反紅功能函數
    for x,y in zip(x_cols,y_cols):
        add_word_color(df,x,y)
    
    # 存檔
    writer.save()

In [4]:
for fname in tqdm(sorted([ i for i in df['from'].unique()])):
    d = df.loc[df['from']==fname]
    save_name = '../data/測試結果/'+fname[:4]+'測試結果.xlsx'
    save_color_df(d,save_name)
    print(save_name,d.shape)
    

  5%|▍         | 2/44 [00:00<00:04,  8.78it/s]

../data/測試結果/0927測試結果.xlsx (34, 47)
../data/測試結果/0928測試結果.xlsx (56, 47)


 11%|█▏        | 5/44 [00:00<00:03, 10.72it/s]

../data/測試結果/0929測試結果.xlsx (50, 47)
../data/測試結果/1004測試結果.xlsx (34, 47)
../data/測試結果/1005測試結果.xlsx (30, 47)


 16%|█▌        | 7/44 [00:00<00:03, 10.22it/s]

...

../data/測試結果/1203測試結果.xlsx (44, 47)
../data/測試結果/1206測試結果.xlsx (33, 47)


 98%|█████████▊| 43/44 [00:04<00:00,  9.01it/s]

../data/測試結果/1207測試結果.xlsx (46, 47)
../data/測試結果/1208測試結果.xlsx (39, 47)
../data/測試結果/1209測試結果.xlsx (36, 47)


100%|██████████| 44/44 [00:04<00:00,  9.42it/s]

../data/測試結果/1210測試結果.xlsx (46, 47)
